<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 4*

# 📝 Assignment
---

# 모델선택(Model Selection)

### 1) 캐글 대회를 이어서 진행합니다. RandomizedSearchCV 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.

- [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)를 사용하세요.
- 분류문제에서 맞는 [scoring parameter](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values) metric을 사용하세요.
- [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) 사용을 권합니다.
- RandomizedSearchCV 를 사용해서 하이퍼파라미터 튜닝을 진행하고 최고 성능을 보이는 모델로 예측을 진행한 후 캐글에 제출합니다.
- **캐글 Leaderboard에서 개선된 본인 Score를 과제 제출폼에 제출하세요.**

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# !pip install kaggle

# ! pip install category_encoders

In [3]:
import pandas as pd
import numpy as np

In [4]:
### EDA 시작
target = 'vacc_h1n1_f'
# target = 'vacc_seas_f'
train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [6]:
# 타겟과 관련 없는 특성 제거하기
col_seas = ['doctor_recc_seasonal', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc']

train = train.drop(col_seas, axis=1)
test = test.drop(col_seas, axis=1)

In [13]:
# 결측치 많은 특성 제거하기
col_toomany_nan = ['employment_industry', 'employment_occupation']

train = train.drop(col_toomany_nan, axis=1)
test = test.drop(col_toomany_nan, axis=1)

In [14]:
print(train.shape)
train.isna().sum()

(42154, 33)


h1n1_concern                    8420
h1n1_knowledge                  8420
behavioral_antiviral_meds       8519
behavioral_avoidance            8672
behavioral_face_mask            8444
behavioral_wash_hands           8471
behavioral_large_gatherings     8514
behavioral_outside_home         8521
behavioral_touch_face           8583
doctor_recc_h1n1                1885
chronic_med_condition           1317
child_under_6_months            9449
health_insurance               19421
health_worker                   9430
opinion_h1n1_vacc_effective     8779
opinion_h1n1_risk               8825
opinion_h1n1_sick_from_vacc     8869
agegrp                             0
education_comp                 10166
raceeth4_i                         0
sex_i                              0
inc_pov                            0
marital                        10199
rent_own_r                      2196
employment_status              10278
census_region                      0
census_msa                         0
n

In [15]:
train.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,chronic_med_condition,child_under_6_months,health_insurance,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,agegrp,education_comp,raceeth4_i,sex_i,inc_pov,marital,rent_own_r,employment_status,census_region,census_msa,n_adult_r,household_children,n_people_r,hhs_region,state,vacc_h1n1_f
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,10 - 17 Years,NaN,3,1,2,NaN,2.0,NaN,4,"MSA, Not Principle City",2.0,3.0,5.0,9,CALIFORNIA,0
1,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,Very Effective,Very Low,Not Very Worried,45 - 54 Years,3.0,3,1,2,2.0,1.0,Employed,2,"MSA, Principle City",1.0,0.0,1.0,7,NEBRASKA,0
2,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,Somewhat Effective,Somewhat Low,Very Worried,45 - 54 Years,3.0,3,1,2,2.0,2.0,Employed,1,"MSA, Not Principle City",1.0,2.0,3.0,1,MASSACHUSETTS,0
3,2.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,Very Effective,Very Low,Not Very Worried,35 - 44 Years,4.0,1,2,1,1.0,1.0,Employed,3,"MSA, Not Principle City",2.0,2.0,4.0,6,LOUISIANA,0
4,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,Very Effective,Very Low,Not At All Worried,55 - 64 Years,3.0,3,2,2,1.0,1.0,Employed,3,"MSA, Not Principle City",3.0,0.0,3.0,4,ALABAMA,1


In [16]:
def engineer(df):        
    # behavioral_* 통합
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    # 범주형의 값을 순서형 수치로 변경
    df.opinion_h1n1_vacc_effective = df.opinion_h1n1_vacc_effective.replace({'Not At All Effective':0, 'Not Very Effective':1, 
         'Dont Know':2, 'Somewhat Effective':3, 'Very Effective':4, 
         'Refused':np.nan, np.nan:0}
    )
    df.opinion_h1n1_risk = df.opinion_h1n1_risk.replace({'Very Low':0, 'Somewhat Low':1, 'Dont Know':2, 
         'Somewhat High':3, 'Very High':4, 'Refused':np.nan, np.nan:0}
    )
    df.opinion_h1n1_sick_from_vacc = df.opinion_h1n1_sick_from_vacc.replace({'Not At All Worried':0, 'Not Very Worried':1, 'Dont Know':2, 
         'Somewhat Worried':3, 'Very Worried':4, 'Refused':np.nan, np.nan:0}
    )
    df.agegrp = df.agegrp.replace({'6 Months - 9 Years':0, '10 - 17 Years':1, '18 - 34 Years':2, 
         '35 - 44 Years':3, '45 - 54 Years':4, '55 - 64 Years':5, '65+ Years':6}
    )
    df.census_msa = df.census_msa.replace({'Non-MSA':0, 'MSA, Not Principle City':1, 'MSA, Principle City':2})
        
    return df


train = engineer(train)
test = engineer(test)

In [17]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_test = test[features]

### EDA 끝

In [18]:
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder, TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint, uniform

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [43]:
%%time
pipe = make_pipeline(
    OrdinalEncoder(handle_missing='return_nan'), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=2)
)

dists = {
    'randomforestclassifier__n_estimators': randint(150, 300),
    'randomforestclassifier__max_depth': randint(5, 15),
    'randomforestclassifier__max_features': uniform(0, 0.5)
}

clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='roc_auc',  
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train, y_train);

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 12.8min finished


CPU times: user 17.4 s, sys: 694 ms, total: 18.1 s
Wall time: 12min 56s


In [46]:
print('최적 하이퍼파라미터: ', clf.best_params_)
print('roc_auc: ', clf.best_score_)

최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 12, 'randomforestclassifier__max_features': 0.23696472717478695, 'randomforestclassifier__n_estimators': 218}
roc_auc:  0.8433190515372058


In [ ]:
# 최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__max_features': 0.8485466648445711, 'randomforestclassifier__n_estimators': 166}
# F1:  0.569834747536281

# 최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 11, 'randomforestclassifier__max_features': 0.7563127746684387, 'randomforestclassifier__n_estimators': 169}
# F1:  0.5699558806251526

# 최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 13, 'randomforestclassifier__max_features': 0.551813562377767, 'randomforestclassifier__n_estimators': 163}
# F1:  0.5714311138139805

# 최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 12, 'randomforestclassifier__max_features': 0.5383281712274792, 'randomforestclassifier__n_estimators': 184}
# F1:  0.5723167616525429

# 최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__max_features': 0.18997888246058203, 'randomforestclassifier__n_estimators': 260}
# roc_auc:  0.8433578886115308

# 최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__max_features': 0.24737153855180405, 'randomforestclassifier__n_estimators': 193}
# roc_auc:  0.8438301351187466

# 최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 12, 'randomforestclassifier__max_features': 0.23696472717478695, 'randomforestclassifier__n_estimators': 218}
# roc_auc:  0.8433190515372058

In [47]:
# 최고 성능 모델 불러오기
pipe = clf.best_estimator_

# pipe = make_pipeline(
#     OrdinalEncoder(handle_missing="return_nan"), 
#     SimpleImputer(), 
#     RandomForestClassifier(max_depth=12, max_features=0.5383281712274792, n_estimators=184, random_state=2, n_jobs=-1)
# )

# pipe.fit(X_train, y_train)

In [48]:
# 테스트 셋 예측 확률 얻기
y_pred_proba = pipe.predict_proba(X_train)[:, 1] # 타겟이 1(True)일 확률 가져오기

In [49]:
# ROC curve 확인하기
from sklearn.metrics import roc_curve

# roc_curve(타겟값, prob of 1)
fpr, tpr, thresholds = roc_curve(y_train, y_pred_proba)

# threshold 최대값의 인덱스, np.argmax()
optimal_idx = np.argmax(tpr - fpr) # 재현율은 최대로, 위양성률은 최소로 하는 데이터의 인덱스 값 찾기
optimal_threshold = thresholds[optimal_idx] # 해당 값의 임계값 가져오기

print('idx:', optimal_idx, ', threshold:', optimal_threshold)

idx: 4055 , threshold: 0.27828914332889654


In [50]:
# 최적 임계값 적용
y_test_pred_proba = pipe.predict_proba(X_test)[:, 1] # 타겟이 1(True)일 확률 가져오기
y_test_pred = y_test_pred_proba > optimal_threshold

In [51]:
# 최종 결과 담은 데이터프레임 생성
d = {'Id':X_test.index, 'vacc_h1n1_f': y_test_pred}
submission = pd.DataFrame(data = d)
submission.head()

,Id,vacc_h1n1_f
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False


In [52]:
# 제출용 파일 생성
# submission.to_csv('submission.csv', index=False)

In [ ]:
# !kaggle competitions submit -c prediction-of-h1n1-vaccination -f submission.csv -m "Gyeong ho Ahn, Fifth Submission(using Random Forest) using RandomizedSearchCV & threshold"

## 🔥 도전과제


### 2) [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.
- 모델 성능을 높이기 위해 가능한 시도를 다 해보세요.
- 모델 성능에 가장 큰 영향을 준 하이퍼파라미터에 대해서 분석하고 설명해 보세요.



In [29]:
from sklearn.model_selection import GridSearchCV

pipe2 = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=2)
)

params = {
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__n_estimators': [50, 100, 150, 200, 250],
    'randomforestclassifier__max_features': ['auto', 'sqrt', 'log2']
}

clf2 = GridSearchCV(
    pipe, 
    param_grid=params, 
    cv=3, 
    scoring='roc_auc',  
    verbose=1,
    n_jobs=-1
)

clf2.fit(X_train, y_train);

Fitting 3 folds for each of 75 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.4min
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed: 13.7min finished


In [30]:
print('최적 하이퍼파라미터: ', clf2.best_params_)
print('F1: ', clf2.best_score_)

최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__n_estimators': 250}
F1:  0.8429052039628533


In [31]:
pipe2 = clf2.best_estimator_

In [32]:
# 테스트 셋 예측 확률 얻기
y_pred_proba = pipe2.predict_proba(X_train)[:, 1] # 타겟이 1(True)일 확률 가져오기

In [33]:
# ROC curve 확인하기
# roc_curve(타겟값, prob of 1)
fpr, tpr, thresholds = roc_curve(y_train, y_pred_proba)

# threshold 최대값의 인덱스, np.argmax()
optimal_idx = np.argmax(tpr - fpr) # 재현율은 최대로, 위양성률은 최소로 하는 데이터의 인덱스 값 찾기
optimal_threshold = thresholds[optimal_idx] # 해당 값의 임계값 가져오기

print('idx:', optimal_idx, ', threshold:', optimal_threshold)

idx: 5286 , threshold: 0.2643924002598367


In [34]:
# 최적 임계값 적용
y_test_pred_proba = pipe.predict_proba(X_test)[:, 1] # 타겟이 1(True)일 확률 가져오기
y_test_pred = y_test_pred_proba > optimal_threshold

In [35]:
# 최종 결과 담은 데이터프레임 생성
d = {'Id':X_test.index, 'vacc_h1n1_f': y_test_pred}
submission = pd.DataFrame(data = d)
submission.head()

,Id,vacc_h1n1_f
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False


In [36]:
# 제출용 파일 생성
# submission.to_csv('submission.csv', index=False)

In [37]:
# !kaggle competitions submit -c prediction-of-h1n1-vaccination -f submission.csv -m "Gyeong ho Ahn, Fifth Submission(using Random Forest) using GridSearchCV & threshold"

100% 310k/310k [00:00<00:00, 1.47MB/s]
Successfully submitted to Prediction of H1N1 vaccination